In [ ]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
!cp /kaggle/input/backup-806/util_openbook.py .

In [ ]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

In [ ]:
import numpy as np
import pandas as pd 
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import os
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
from datasets import Dataset
from torch.utils.data import DataLoader
import gc

In [ ]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [ ]:
# 定义一个函数，将列表分割成指定大小的子列表
def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

# 定义一个函数，从验证数据集中检索相关的文档
def get_relevant_documents_parsed(df_valid):
    df_chunk_size=600  # 设置每个数据块的大小
    # 从指定路径加载预处理过的数据集
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    # 修改文本，将标题、段落和文本合并，并替换换行符和单引号
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]

    # 初始化两个列表，用于存储文章的索引和值
    all_articles_indices = []
    all_articles_values = []
    # 使用 tqdm 进度条，遍历验证数据集，每次处理一个数据块
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]  # 获取当前数据块
    
        # 调用 retrieval 函数，获取相关文章的索引和得分
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        # 将结果添加到列表中
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    # 将所有结果连接成数组
    article_indices_array = np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    # 获取每个查询的文章数量
    top_per_query = article_indices_array.shape[1]
    # 展平文章信息，并按查询分组
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    # 使用 SplitList 函数，将文章信息分割成与每个查询匹配的组
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles  # 返回检索到的文章列表


# 定义一个函数，从验证数据集中检索相关的文档
def get_relevant_documents(df_valid):
    df_chunk_size=800  # 设置每个数据块的大小
    
    # 从指定路径加载预处理过的数据集
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    # 修改文本，将标题和文本合并，并进行Unicode标准化（NFKD），同时替换双引号
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    # 初始化两个列表，用于存储文章的索引和值
    all_articles_indices = []
    all_articles_values = []
    # 使用 tqdm 进度条，遍历验证数据集，每次处理一个数据块
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]  # 获取当前数据块
    
        # 调用 retrieval 函数，获取相关文章的索引和得分
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        # 将结果添加到列表中
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    # 将所有结果连接成数组
    article_indices_array = np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    # 获取每个查询的文章数量
    top_per_query = article_indices_array.shape[1]
    # 展平文章信息，并按查询分组
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    # 使用 SplitList 函数，将文章信息分割成与每个查询匹配的组
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles  # 返回检索到的文章列表



# 定义一个检索函数，输入为验证数据集和修改过的文本
def retrieval(df_valid, modified_texts):
    # 将验证集的每行合并成一个字符串，包括问题和答案选项
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    # 初始化第一个TF-IDF向量化器
    vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words)
    # 在验证集文本上拟合第一个向量化器
    vectorizer1.fit(corpus_df_valid)
    # 获取验证集的词汇表
    vocab_df_valid = vectorizer1.get_feature_names_out()
    # 使用验证集的词汇表初始化第二个TF-IDF向量化器
    vectorizer = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid)
    # 在修改过的文本上拟合第二个向量化器
    vectorizer.fit(modified_texts[:500000])
    # 转换验证集文本为TF-IDF向量
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
    # 打印向量化器词汇表的长度
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000  # 设置每个数据块的大小
    top_per_chunk = 15   # 设置每个数据块的顶部文档数
    top_per_query = 15   # 设置每个查询的顶部文档数

    # 初始化列表以保存每个数据块的顶部索引和值
    all_chunk_top_indices = []
    all_chunk_top_values = []

    # 使用 tqdm 进度条，遍历修改过的文本，每次处理一个数据块
    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        # 将当前数据块的文本转换为TF-IDF向量
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        # 计算验证集和当前数据块的文档之间的余弦相似度
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        # 获取每个查询的顶部文档索引和值
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        # 将结果添加到列表中
        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    # 将所有结果连接成数组
    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    # 获取每个查询的顶部文档得分和索引
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    # 获取文章的索引
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    # 返回文章的索引和得分
    return articles_indices, merged_top_scores


# 函数定义：准备用于模型的问答输入
def prepare_answering_input(
        tokenizer,        # 令牌化器，用于将文本转换为令牌
        question,         # 提供的问题字符串
        options,          # 提供的选项列表
        context,          # 提供的上下文字符串
        max_seq_length=4096, # 最大序列长度，默认为4096
    ):
    # 拼接上下文、令牌化器的起始令牌和问题
    c_plus_q   = context + ' ' + tokenizer.bos_token + ' ' + question
    # 创建一个列表，包含与选项数量相同的拼接字符串
    c_plus_q_4 = [c_plus_q] * len(options)
    # 使用令牌化器处理拼接字符串和选项，生成令牌化的示例
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,  # 指定最大长度
        padding="longest",          # 填充至最长示例的长度
        truncation=False,           # 不截断
        return_tensors="pt",        # 返回PyTorch张量
    )
    # 从令牌化的示例中获取输入ID，并增加一个维度
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    # 从令牌化的示例中获取注意力掩码，并增加一个维度
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    # 创建一个字典，包含输入ID和注意力掩码，同时将它们移动到模型的设备上
    example_encoded = {
        "input_ids": input_ids.to(model.device.index),
        "attention_mask": attention_mask.to(model.device.index),
    }
    # 返回编码后的示例
    return example_encoded

In [ ]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']

In [ ]:
df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")

In [ ]:
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()

In [ ]:
retrieved_articles = get_relevant_documents(df_valid)
gc.collect()

In [ ]:
tokenizer_dir = "/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
config = torch.load("/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/config.pth")
model = AutoModelForMultipleChoice.from_config(config)

state = torch.load(f"/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model2/pytorch_model.bin",
                       map_location=torch.device('cpu'))
model.load_state_dict(state)
model = model.cuda()
model.eval()

In [ ]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_article[-15][2]}\n{retrieved_article[-14][2]}\n{retrieved_article[-13][2]}\n{retrieved_article[-12][2]}\n{retrieved_article[-11][2]}\n{retrieved_article[-10][2]}\n{retrieved_article[-9][2]}\n{retrieved_article[-8][2]}\n{retrieved_article[-7][2]}\n{retrieved_article[-6][2]}\n{retrieved_article[-5][2]}\n{retrieved_article[-4][2]}\n{retrieved_article[-3][2]}\n{retrieved_article[-2][2]}\n{retrieved_article[-1][2]}" for retrieved_article in retrieved_articles]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-4820:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'

# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsj = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsj.append(outputs.logits.cpu().detach())

test_predictionsj = torch.cat(test_predictionsj)

In [ ]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_articles_parse[-15][2]}\n{retrieved_articles_parse[-14][2]}\n{retrieved_articles_parse[-13][2]}\n{retrieved_articles_parse[-12][2]}\n{retrieved_articles_parse[-11][2]}\n{retrieved_articles_parse[-10][2]}\n{retrieved_articles_parse[-9][2]}\n{retrieved_articles_parse[-8][2]}\n{retrieved_articles_parse[-7][2]}\n{retrieved_articles_parse[-6][2]}\n{retrieved_articles_parse[-5][2]}\n{retrieved_articles_parse[-4][2]}\n{retrieved_articles_parse[-3][2]}\n{retrieved_articles_parse[-2][2]}\n{retrieved_articles_parse[-1][2]}" for retrieved_articles_parse in retrieved_articles_parsed]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-4820:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsi = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsi.append(outputs.logits.cpu().detach())

test_predictionsi = torch.cat(test_predictionsi)

In [ ]:
model_dir = "/kaggle/input/llm-fintune3/LLM_fintune3/LLM_model1"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

In [ ]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_article[-15][2]}\n{retrieved_article[-14][2]}\n{retrieved_article[-13][2]}\n{retrieved_article[-12][2]}\n{retrieved_article[-11][2]}\n{retrieved_article[-10][2]}\n{retrieved_article[-9][2]}\n{retrieved_article[-8][2]}\n{retrieved_article[-7][2]}\n{retrieved_article[-6][2]}\n{retrieved_article[-5][2]}\n{retrieved_article[-4][2]}\n{retrieved_article[-3][2]}\n{retrieved_article[-2][2]}\n{retrieved_article[-1][2]}" for retrieved_article in retrieved_articles]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-4820:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'

# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsk = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsk.append(outputs.logits.cpu().detach())

test_predictionsk = torch.cat(test_predictionsk)

In [ ]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df['context'] = [f"{retrieved_articles_parse[-15][2]}\n{retrieved_articles_parse[-14][2]}\n{retrieved_articles_parse[-13][2]}\n{retrieved_articles_parse[-12][2]}\n{retrieved_articles_parse[-11][2]}\n{retrieved_articles_parse[-10][2]}\n{retrieved_articles_parse[-9][2]}\n{retrieved_articles_parse[-8][2]}\n{retrieved_articles_parse[-7][2]}\n{retrieved_articles_parse[-6][2]}\n{retrieved_articles_parse[-5][2]}\n{retrieved_articles_parse[-4][2]}\n{retrieved_articles_parse[-3][2]}\n{retrieved_articles_parse[-2][2]}\n{retrieved_articles_parse[-1][2]}" for retrieved_articles_parse in retrieved_articles_parsed]

test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[-4820:]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
test_predictionsl = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictionsl.append(outputs.logits.cpu().detach())

test_predictionsl = torch.cat(test_predictionsl)

In [ ]:
test_predictions = (test_predictionsi+test_predictionsj+test_predictionsk+test_predictionsl)/4.0

In [ ]:
predictions_as_ids = np.argsort(-test_predictions, 1)

predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
# predictions_as_answer_letters[:3]

predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)